In [1]:
%pip install python-dotenv psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import psycopg2
from dotenv import load_dotenv
# ... (other imports)

# load_dotenv()

# USER=os.getenv("USER")
# PASSWORD=os.getenv("PASSWORD")
# HOST=os.getenv("HOST")
# PORT=5432
# DBNAME=os.getenv("DBNAME")

USER="postgres" 
PASSWORD = "itspgpassword@2025"
HOST = "db.kcsdqmtsdasqlancgygk.supabase.co" # Use the correct format here
PORT="5432"
DBNAME = "postgres"



# Connect to the database
try:
    connection = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME,
        sslmode='require' 
    )
    print("Connection successful! Supabase credentials validated.")

except Exception as e:
    # If this fails, the issue is still a typo in HOST or a network firewall blocking port 5432.
    print(f"Failed to connect: {e}")

Connection successful! Supabase credentials validated.


In [3]:
cur = connection.cursor()

In [4]:
QUERY = "SELECT product_name, price, category FROM product_details LIMIT 3;" 
    
# 3. Execute the query
cur.execute(QUERY)
# 4. Fetch all results
results = cur.fetchall()

In [5]:
results

[("Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2",
  '257',
  'Personal Care'),
 ('Care Cosmetics Apple Day Cream with Apple juice to improve skin complexion - 50gm - Pack of 2',
  '386',
  'Personal Care'),
 ("Care Cosmetics Care's Chicknut Face Pack with Chikoo for sensitive skin - 60gm - Pack of 2",
  '460',
  'Personal Care')]

In [6]:
connection.commit()

In [7]:
connection.rollback()

In [8]:
cur.execute("select id,product_name,category,price,rating,available from product_details LIMIT 3;")
results = cur.fetchall()
connection.commit()
results

[(1,
  "Care Cosmetics Care's Antidandruff shampoo with Punica Granatum Peel Extract - 100ml -Pack of 2",
  'Personal Care',
  '257',
  'N/A',
  'yes'),
 (2,
  'Care Cosmetics Apple Day Cream with Apple juice to improve skin complexion - 50gm - Pack of 2',
  'Personal Care',
  '386',
  'N/A',
  'no'),
 (3,
  "Care Cosmetics Care's Chicknut Face Pack with Chikoo for sensitive skin - 60gm - Pack of 2",
  'Personal Care',
  '460',
  'N/A',
  'yes')]

In [9]:
type(results[0])

tuple

In [10]:
%pip install langchain langchain-google-genai langchain-community sqlalchemy python-dotenv

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
# Install necessary packages if not already installed
# Uncomment if needed:
# !pip install langchain langchain-google-genai langchain-community sqlalchemy python-dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

c:\Users\kumar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Load environment variables (optional if using .env) ---
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_DB_URL = os.getenv("SUPABASE_DB_URL")

In [3]:
# --- Initialize the Gemini model ---
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  
    api_key=GEMINI_API_KEY,
    temperature=0.0
)

In [4]:
llm.invoke("Hello").content

'Hello! How can I help you today?'

In [5]:
try:
    engine = create_engine(SUPABASE_DB_URL)
    db = SQLDatabase(
        engine=engine,
        include_tables=['product_details']  # only this table for now
    )
    print("✅ Database connection successful.")
except Exception as e:
    print(f"❌ Error connecting to DB: {e}")


✅ Database connection successful.


In [6]:
CUSTOM_SYSTEM_PREFIX = """
You are an intelligent Text-to-SQL + Knowledge agent for a personal care products company.
You have access to a PostgreSQL database that stores structured product information like name, category, price, and availability.

Your job is to:
1. Convert product-related questions into accurate SQL queries, execute them, and present clear, human-like answers.
2. If the product or requested data is NOT found in the database, you must switch to your general world knowledge or perform an internet-style reasoning search to answer naturally — do NOT say that you lack information or cannot answer.
3. For example, if a user asks about the **benefits, uses, or effects** of a product that’s not present in the database, give a helpful and factual general response based on your own knowledge or common facts available online.
4. Only use the database for factual, structured data (like price, category, stock availability, etc.).
5. When switching to general knowledge mode, never generate SQL queries.
6. Always be polite, conversational, and confident in your answers.
7. Limit database query results to 10 rows maximum.
8. Never use INSERT, UPDATE, DELETE, or DROP commands.

**Summary of Behavior:**
- If the query is about structured product data → Use SQL.
- If the query is about general info, benefits, usage, or product advice → Use your internal knowledge or online reasoning.
- Never say: “I cannot directly tell you…” or “I don’t have information.”
- Always return a friendly, natural-sounding response.

Begin!
"""



In [7]:
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=CUSTOM_SYSTEM_PREFIX,
    verbose=True,
    handle_parsing_errors=True
)
print("✅ Agent successfully created.")

✅ Agent successfully created.


In [8]:
def get_product_info(user_query: str):
    """Executes Text-to-SQL flow for user queries."""
    print(f"\n--- Processing User Query: '{user_query}' ---")
    try:
        response = agent_executor.invoke({"input": user_query})
        return response['output']
    except Exception as e:
        return f"An error occurred during query execution: {e}"


In [14]:
user_question_1 = "whats the benifit of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant."
answer_1 = get_product_info(user_question_1)
print(f"\n💬 Chatbot Response:\n{answer_1}")



--- Processing User Query: 'whats the benifit of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant.' ---


> Entering new SQL Agent Executor chain...

💬 Chatbot Response:
An error occurred during query execution: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `That's a great question! Medibar Personal Lubricant, like many personal lubricants, is generally designed to enhance comfort and pleasure during intimate activities.

Here are some common benefits of using a personal lubricant:

*   **Reduces Friction:** It helps to minimize friction, which can prevent discomfort, irritation, or even minor injuries during intimacy.
*   **Enhances Pleasure:** By providing a smooth glide, it can make intimate experiences more enjoyable and comfortable for both partners.
*   **Addresses Dryness:** It's particul

In [15]:
print(f"\n💬 Chatbot Response:\n{answer_1}")


💬 Chatbot Response:
An error occurred during query execution: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `That's a great question! Medibar Personal Lubricant, like many personal lubricants, is generally designed to enhance comfort and pleasure during intimate activities.

Here are some common benefits of using a personal lubricant:

*   **Reduces Friction:** It helps to minimize friction, which can prevent discomfort, irritation, or even minor injuries during intimacy.
*   **Enhances Pleasure:** By providing a smooth glide, it can make intimate experiences more enjoyable and comfortable for both partners.
*   **Addresses Dryness:** It's particularly helpful for individuals experiencing natural dryness, which can be caused by various factors like stress, medication, hormonal changes, or certain medical conditions.
*   **Compa

In [17]:
system = "I will give you a query and the chat response in which there will be unwanted text in the response unrelated to the query. Your task is to clean the response and provide only the relevant answer to the query without any unrelated text and without any unwanted symbols like ** or * ."
llm.invoke(system+answer_1).content

'That\'s a great question! Medibar Personal Lubricant, like many personal lubricants, is generally designed to enhance comfort and pleasure during intimate activities.\n\nHere are some common benefits of using a personal lubricant:\n\n*   Reduces Friction: It helps to minimize friction, which can prevent discomfort, irritation, or even minor injuries during intimacy.\n*   Enhances Pleasure: By providing a smooth glide, it can make intimate experiences more enjoyable and comfortable for both partners.\n*   Addresses Dryness: It\'s particularly helpful for individuals experiencing natural dryness, which can be caused by various factors like stress, medication, hormonal changes, or certain medical conditions.\n*   Compatible with Toys: Many lubricants are safe to use with sex toys, making playtime more comfortable and fun.\n*   Long-Lasting Formulas: Products like the Medibar lubricant, which are described as "long-lasting," are formulated to maintain their slipperiness for an extended pe

In [18]:
print(llm.invoke(system+answer_1).content)

That's a great question! Medibar Personal Lubricant, like many personal lubricants, is generally designed to enhance comfort and pleasure during intimate activities.

Here are some common benefits of using a personal lubricant:

*   Reduces Friction: It helps to minimize friction, which can prevent discomfort, irritation, or even minor injuries during intimacy.
*   Enhances Pleasure: By providing a smooth glide, it can make intimate experiences more enjoyable and comfortable for both partners.
*   Addresses Dryness: It's particularly helpful for individuals experiencing natural dryness, which can be caused by various factors like stress, medication, hormonal changes, or certain medical conditions.
*   Compatible with Toys: Many lubricants are safe to use with sex toys, making playtime more comfortable and fun.
*   Long-Lasting Formulas: Products like the Medibar lubricant, which are described as "long-lasting," are formulated to maintain their slipperiness for an extended period, reduc

In [23]:
llm.invoke("whats the benifit of using Medibar Personal Lubricant for Men & Women, Long Lasting Pleasure Lubricant").content

'Medibar Personal Lubricant, especially with its "Long Lasting Pleasure" claim, offers several key benefits for both men and women:\n\n1.  **Enhanced Comfort and Reduced Friction:**\n    *   **Prevents Discomfort:** The primary benefit of any lubricant is to reduce friction during sexual activity. This prevents chafing, irritation, and soreness that can occur due to insufficient natural lubrication.\n    *   **Smoother Movement:** It allows for more fluid and comfortable movement, making penetration easier and more enjoyable for both partners.\n\n2.  **Combats Natural Dryness:**\n    *   **Addresses Vaginal Dryness:** Many women experience vaginal dryness due to various factors like hormonal changes (menopause, postpartum), certain medications, stress, or even just insufficient arousal. Medibar helps to supplement natural lubrication, making intercourse comfortable and pleasurable again.\n    *   **For Men:** While less common, some men can also benefit from lubrication, especially dur

In [19]:
user_question_2 = "Who is elon musk"
answer_2 = get_product_info(user_question_2)
system = "I will give you a query and the chat response in which there will be unwanted text in the response unrelated to the query. Your task is to clean the response and provide only the relevant answer to the query without any unrelated text and without any unwanted symbols like ** or * ."
print(llm.invoke(system+answer_2).content)


--- Processing User Query: 'Who is elon musk' ---


> Entering new SQL Agent Executor chain...
Elon Musk is a prominent entrepreneur and business magnate. He is the founder, CEO, and chief engineer of SpaceX; angel investor, CEO, and product architect of Tesla, Inc.; owner, chairman, and CTO of X (formerly Twitter); and founder of The Boring Company, xAI, and co-founder of Neuralink and OpenAI. He's known for his ambitious projects in electric vehicles, space exploration, and artificial intelligence.
